In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from urllib.parse import quote
import random
import re

# 기업 리스트
companies = ["삼성중공업"]

# 날짜 리스트
dates = pd.date_range(start="2020-01-01", end="2024-12-31").strftime("%Y.%m.%d").tolist()

# 결과 저장용
results = []
seen = set()

# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 크롤링 시작
for company in companies:
    print(f"\n🔍 [기업 시작] {company}")
    encoded_query = quote(company)

    for target_date in dates:
        print(f"   📅 [날짜 처리 중] {target_date}")
        display_date = target_date.replace(".", "")
        count_per_day = 0

        for start in range(1, 1000, 10):
            url = (
                f"https://search.naver.com/search.naver?where=news&query={encoded_query}"
                f"&sort=0&ds={target_date}&de={target_date}"
                f"&nso=so:r,p:from{display_date}to{display_date},a:all"
                f"&start={start}"
            )

            driver.get(url)
            time.sleep(random.uniform(2, 4))  # 페이지 로딩 대기

            try:
                # 뉴스 기사 링크는 보통 외부 링크이며 target="_blank"를 가짐
                news_elements = driver.find_elements(By.CSS_SELECTOR, 'a[target="_blank"]')
            except Exception as e:
                print(f"      ⚠️ 구조 탐색 실패 - {company} / {target_date} - {e}")
                break

            if not news_elements:
                break  # 뉴스 없으면 다음 날짜로

            for elem in news_elements:
                try:
                    href = elem.get_attribute("href")
                    if not href or "news" not in href:
                        continue  # 뉴스 관련 URL만 필터링

                    # 자식 span 태그에서 제목 추출
                    try:
                        span = elem.find_element(By.TAG_NAME, "span")
                        raw_title = span.get_attribute("textContent").strip()
                    except:
                        continue

                    # 제목 전처리
                    cleaned_title = re.sub(r'\[.*?\]', '', raw_title).strip()
                    cleaned_title = cleaned_title.replace(company, "").strip(", ").strip()

                    if not cleaned_title:
                        continue

                    if (company, cleaned_title, href) in seen:
                        continue
                    seen.add((company, cleaned_title, href))

                    results.append({
                        "기업명": company,
                        "날짜": target_date,
                        "뉴스제목": cleaned_title,
                        "URL": href
                    })
                    count_per_day += 1

                except Exception as e:
                    print(f"      ⚠️ 추출 오류: {e}")
                    continue

            if count_per_day == 0:
                break  # 해당 페이지 이후엔 더 이상 뉴스 없음

        if count_per_day > 0:
            print(f"   ✅ [수집 완료] {target_date} - {count_per_day}건")

        time.sleep(random.uniform(1, 2))  # 날짜 간 대기

    print("🌙 [대기] 다음 기업 전환 전 10초 대기...")
    time.sleep(10)

# 드라이버 종료
driver.quit()

# 저장
output_filename = f"{companies[0]}_뉴스.csv"
df = pd.DataFrame(results)
df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n📁 전체 완료! 총 {len(df)}건의 뉴스가 저장되었습니다: {output_filename}")



🔍 [기업 시작] 삼성중공업
   📅 [날짜 처리 중] 2020.01.01
   ✅ [수집 완료] 2020.01.01 - 47건
   📅 [날짜 처리 중] 2020.01.02
   ✅ [수집 완료] 2020.01.02 - 45건
   📅 [날짜 처리 중] 2020.01.03
   ✅ [수집 완료] 2020.01.03 - 54건
   📅 [날짜 처리 중] 2020.01.04
   ✅ [수집 완료] 2020.01.04 - 4건
   📅 [날짜 처리 중] 2020.01.05
   ✅ [수집 완료] 2020.01.05 - 32건
   📅 [날짜 처리 중] 2020.01.06
   ✅ [수집 완료] 2020.01.06 - 54건
   📅 [날짜 처리 중] 2020.01.07
   ✅ [수집 완료] 2020.01.07 - 48건
   📅 [날짜 처리 중] 2020.01.08
   ✅ [수집 완료] 2020.01.08 - 46건
   📅 [날짜 처리 중] 2020.01.09
   ✅ [수집 완료] 2020.01.09 - 66건
   📅 [날짜 처리 중] 2020.01.10
   ✅ [수집 완료] 2020.01.10 - 116건
   📅 [날짜 처리 중] 2020.01.11
   ✅ [수집 완료] 2020.01.11 - 19건
   📅 [날짜 처리 중] 2020.01.12
   ✅ [수집 완료] 2020.01.12 - 40건
   📅 [날짜 처리 중] 2020.01.13
   ✅ [수집 완료] 2020.01.13 - 43건
   📅 [날짜 처리 중] 2020.01.14
   ✅ [수집 완료] 2020.01.14 - 76건
   📅 [날짜 처리 중] 2020.01.15
   ✅ [수집 완료] 2020.01.15 - 49건
   📅 [날짜 처리 중] 2020.01.16
   ✅ [수집 완료] 2020.01.16 - 229건
   📅 [날짜 처리 중] 2020.01.17
   ✅ [수집 완료] 2020.01.17 - 80건
   📅 [날짜 처리 중] 2020.01.18
   ✅

KeyboardInterrupt: 